<a href="https://colab.research.google.com/github/gyuholee94/NLP/blob/master/%EC%9D%BC%EB%B3%84%EC%9D%84_%EC%9C%84%ED%95%9C_%EB%9D%BC%EB%B2%A8%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 불러오기

In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dbfread import DBF
from haversine import haversine
from tqdm.notebook import tqdm


In [ ]:
%matplotlib inline
import platform
platform.system()
if platform.system() == 'Darwin': # Mac 환경 폰트 설정
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows': # Windows 환경 폰트 설정
    plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False) # 마이너스 폰트 설정
%config InlineBackend.figure_format = 'retina'

In [ ]:
!pip install dbfread
!pip install haversine

# 날짜별 버스 사용하기 위해 ARS라벨링

### 다시 불러오기

In [ ]:
bus_info = pd.read_excel(r'./서울시버스정류소좌표데이터(2020.03.06).xlsx')
train_loc = pd.read_csv(r'./서울 지하철역 좌표.csv')
bus_date = pd.read_csv('./BUS_STATION_BOARDING_MONTH_202002.csv',encoding = 'cp949')

## 라벨링을 위한 기본 전처리

In [ ]:
train_loc

,역명,상세주소,경도,위도,상태
0,서울,서울특별시 중구 봉래동2가 122 서울역(1호선),126.969563,37.558156,정좌표
1,시청,서울특별시 중구 정동 5-5 시청역(1호선),126.976919,37.565597,정좌표
2,종각,서울특별시 종로구 종로1가 54 종각지하철역사,126.981390,37.570222,정좌표
3,종로5가,서울특별시 종로구 종로5가 82-1 1호선 종로5가역,127.001925,37.571020,정좌표
4,동대문,서울특별시 종로구 창신동 492-1 1호선 동대문역(1호선),127.011233,37.571759,정좌표
...,...,...,...,...,...
273,부천종합운동장,경기도 부천시 원미구 춘의동 15-1 부천종합운동장역,126.797466,37.505639,실패
274,춘의,경기도 부천시 원미구 춘의동 145 춘의역,126.787091,37.503859,실패
275,신중동,경기도 부천시 원미구 중동 1140-6 신중동역,126.775952,37.503189,실패
276,부천시청,경기도 부천시 원미구 중동 1243 부천시청역,126.764025,37.504748,실패


In [ ]:
#지하철 위치 전처리(역명, 경도, 위도)
train_loc = train_loc[['역명', '경도', '위도']]
train_loc=train_loc.drop_duplicates(['역명'], keep = 'first')
train_loc.set_index('역명', drop=True, inplace=True)
train_loc

,경도,위도
역명,,
서울,126.969563,37.558156
시청,126.976919,37.565597
종각,126.981390,37.570222
종로5가,127.001925,37.571020
동대문,127.011233,37.571759
...,...,...
부천종합운동장,126.797466,37.505639
춘의,126.787091,37.503859
신중동,126.775952,37.503189


In [ ]:
bus_col = bus_info.columns
bus_col

Index(['표준ID', 'ARS-ID', '정류장명', 'X좌표', 'Y좌표', '비고'], dtype='object')

In [ ]:
bus_info_need = bus_info[['ARS-ID', '정류장명', 'X좌표', 'Y좌표']]
bus_info_need.columns =  ['ARS', '역명', '경도', '위도']
bus_info_need=bus_info_need.drop_duplicates(['ARS'], keep = 'first')
bus_info_need.set_index('ARS', drop=True, inplace=True)
bus_info_need

,역명,경도,위도
ARS,,,
1001,종로2가사거리,126.987750,37.569765
1002,창경궁.서울대학교병원,126.996566,37.579183
1003,명륜3가.성대입구,126.998340,37.582671
1004,종로2가.삼일교,126.987613,37.568579
1005,혜화동로터리.여운형활동터,127.001744,37.586243
...,...,...,...
25995,우성아파트,127.139339,37.550386
25996,우성아파트,127.140046,37.550643
25997,조일약국,127.123596,37.533630


## 버스 일별 데이터 전처리

In [ ]:
#버스 일별 데이터 확인
bus_date

,사용일자,노선번호,노선명,버스정류장ARS번호,역명,승차총승객수,하차총승객수,등록일자
0,20200201,100,100번(하계동~용산구청),11428,한성여객종점,14,4,20200204
1,20200201,100,100번(하계동~용산구청),11387,노원평생학습관,7,0,20200204
2,20200201,100,100번(하계동~용산구청),11373,중계역,27,2,20200204
3,20200201,100,100번(하계동~용산구청),11381,중계목화아파트4단지,120,13,20200204
4,20200201,100,100번(하계동~용산구청),11311,인덕대학,172,123,20200204
...,...,...,...,...,...,...,...,...
1118042,20200229,1111,1111번(번동~성북동),08128,KT월곡지사,43,38,20200303
1118043,20200229,1111,1111번(번동~성북동),08130,종암SK아파트,41,73,20200303
1118044,20200229,1111,1111번(번동~성북동),08134,동덕여대앞,75,58,20200303
1118045,20200229,1111,1111번(번동~성북동),08153,종암동주민센터,134,188,20200303


In [ ]:
#역 갯수 확인
len(bus_date['역명'].unique())

8065

In [ ]:
bus_col = bus_date.columns

In [ ]:
bus_col = list(bus_col)
bus_col

['사용일자', '노선번호', '노선명', '버스정류장ARS번호', '역명', '승차총승객수', '하차총승객수', '등록일자']

In [ ]:
#ARS기준으로 묶어줘야 할듯
bus_date_need = bus_date[['사용일자', '버스정류장ARS번호', '역명', '승차총승객수', '하차총승객수']]
bus_date_need.columns =  ['사용일자', 'ARS', '역명', '승차총승객수', '하차총승객수']
bus_date_need

,사용일자,ARS,역명,승차총승객수,하차총승객수
0,20200201,11428,한성여객종점,14,4
1,20200201,11387,노원평생학습관,7,0
2,20200201,11373,중계역,27,2
3,20200201,11381,중계목화아파트4단지,120,13
4,20200201,11311,인덕대학,172,123
...,...,...,...,...,...
1118042,20200229,08128,KT월곡지사,43,38
1118043,20200229,08130,종암SK아파트,41,73
1118044,20200229,08134,동덕여대앞,75,58
1118045,20200229,08153,종암동주민센터,134,188


In [ ]:
bus_date_need['ARS']

0          11428
1          11387
2          11373
3          11381
4          11311
           ...  
1118042    08128
1118043    08130
1118044    08134
1118045    08153
1118046    08151
Name: ARS, Length: 1118047, dtype: object

## 같은날짜, 같은 역의 승하차 총 승객수로 묶어주자. 그러려면 ARS만 가지고 묶고 더하면 사용일자가 뭉개지니까 사용일자, ARS, 역명 기준으로 groupby랑 sum해주기

In [ ]:
bus_date_updown = bus_date_need.groupby(['사용일자', 'ARS']).sum()
bus_date_updown.index

MultiIndex([(20200201, '01001'),
            (20200201, '01002'),
            (20200201, '01003'),
            (20200201, '01004'),
            (20200201, '01005'),
            (20200201, '01006'),
            (20200201, '01007'),
            (20200201, '01008'),
            (20200201, '01009'),
            (20200201, '01010'),
            ...
            (20200229, '68262'),
            (20200229, '68263'),
            (20200229, '68264'),
            (20200229, '68268'),
            (20200229, '68270'),
            (20200229, '68271'),
            (20200229, '68290'),
            (20200229, '68351'),
            (20200229, '68352'),
            (20200229,     '~')],
           names=['사용일자', 'ARS'], length=368013)

## 우선, ARS와 지하철역 거리기준으로 매칭

In [ ]:
bus_date_need

,사용일자,ARS,역명,승차총승객수,하차총승객수
0,20200201,11428,한성여객종점,14,4
1,20200201,11387,노원평생학습관,7,0
2,20200201,11373,중계역,27,2
3,20200201,11381,중계목화아파트4단지,120,13
4,20200201,11311,인덕대학,172,123
...,...,...,...,...,...
1118042,20200229,08128,KT월곡지사,43,38
1118043,20200229,08130,종암SK아파트,41,73
1118044,20200229,08134,동덕여대앞,75,58
1118045,20200229,08153,종암동주민센터,134,188


In [ ]:
bus_info_need

,역명,경도,위도
ARS,,,
1001,종로2가사거리,126.987750,37.569765
1002,창경궁.서울대학교병원,126.996566,37.579183
1003,명륜3가.성대입구,126.998340,37.582671
1004,종로2가.삼일교,126.987613,37.568579
1005,혜화동로터리.여운형활동터,127.001744,37.586243
...,...,...,...
25995,우성아파트,127.139339,37.550386
25996,우성아파트,127.140046,37.550643
25997,조일약국,127.123596,37.533630


In [ ]:
#혹시 에러뜰까봐 일단 이것도 만들어 둠
bus_info_need_ = bus_info_need[['경도', '위도']]
bus_info_need_

,경도,위도
ARS,,
1001,126.987750,37.569765
1002,126.996566,37.579183
1003,126.998340,37.582671
1004,126.987613,37.568579
1005,127.001744,37.586243
...,...,...
25995,127.139339,37.550386
25996,127.140046,37.550643
25997,127.123596,37.533630


In [ ]:
train_loc

,경도,위도
역명,,
서울,126.969563,37.558156
시청,126.976919,37.565597
종각,126.981390,37.570222
종로5가,127.001925,37.571020
동대문,127.011233,37.571759
...,...,...
부천종합운동장,126.797466,37.505639
춘의,126.787091,37.503859
신중동,126.775952,37.503189


In [ ]:
# close_station = {'지하철역' : [],
#                 'ARS' : []}

# for b_station in tqdm(bus_info_need.index):
#     start = (float(bus_info_need.loc[b_station]['위도']), float(bus_info_need.loc[b_station]['경도']))
#     distance_dic = {}
#     for t_station in train_loc.index:
#         goal = (train_loc.loc[t_station]['위도'].mean(), train_loc.loc[t_station]['경도'].mean())
#         distance_dic[t_station] = float(haversine(start, goal))
#     close_ = sorted((value, key) for key, value in distance_dic.items())[0][1]
#     close_station['ARS'].append(b_station)
#     close_station['지하철역'].append(close_)

In [ ]:
# station_mapping = pd.DataFrame(close_station)
# station_mapping = station_mapping[['ARS', '지하철역']]
# station_mapping['ARS']

0         1001
1         1002
2         1003
3         1004
4         1005
         ...  
11274    25995
11275    25996
11276    25997
11277    25998
11278    25999
Name: ARS, Length: 11279, dtype: int64

In [ ]:
# station_mapping.to_csv("busARS_train_station_mapping.csv")

In [ ]:
station_mapping = pd.read_csv('busARS_Train_station_mapping.csv', index_col=0)
station_mapping

,ARS,지하철역
0,1001,종로3가(탑골공원)
1,1002,혜화
2,1003,혜화
3,1004,을지로3가
4,1005,한성대입구(삼선교)
...,...,...
11274,25995,명일
11275,25996,명일
11276,25997,강동구청
11277,25998,천호(풍납토성)


In [ ]:
bus_date_updown.columns

Index(['승차총승객수', '하차총승객수'], dtype='object')

In [ ]:
bus_date_updown.index

MultiIndex([(20200201, '01001'),
            (20200201, '01002'),
            (20200201, '01003'),
            (20200201, '01004'),
            (20200201, '01005'),
            (20200201, '01006'),
            (20200201, '01007'),
            (20200201, '01008'),
            (20200201, '01009'),
            (20200201, '01010'),
            ...
            (20200229, '68262'),
            (20200229, '68263'),
            (20200229, '68264'),
            (20200229, '68268'),
            (20200229, '68270'),
            (20200229, '68271'),
            (20200229, '68290'),
            (20200229, '68351'),
            (20200229, '68352'),
            (20200229,     '~')],
           names=['사용일자', 'ARS'], length=368013)

## 해결해야 하는것
1. ARS가 4자리로 되어있는걸 5자리로 만들어줘야 한다
2. 날짜/ARS로 하니까 인덱스가 겹쳐버리는데, 날짜로만 인덱스 하고 ARS는 그냥 나중에 붙이는 식으로 해야할 듯. 

In [ ]:
ARS_list = []
for ars in list(station_mapping['ARS']):
    if len(ars) == 5:
        pass
    else:
        

SyntaxError: invalid syntax (<ipython-input-22-dbc72f70081e>, line 3)

In [ ]:
group = bus_date_need.groupby(['사용일자', 'ARS']).sum()
group

승차총승객수  하차총승객수
사용일자     ARS                  
20200201 01001     326     446
         01002    1063     859
         01003    3076    1839
         01004     491     440
         01005    1135    2253
...                ...     ...
20200229 68271       7      10
         68290      34       5
         68351      60      12
         68352      11      50
         ~        4095    3568

[368013 rows x 2 columns]

In [ ]:
busARS_train_num = pd.merge(bus_date_updown, station_mapping, how='inner', on='ARS')
busARS_train_num
--> 안된다... ARS도 인덱스에 들어가있어...

(5) index와 Key를 혼합해서 DataFrame 합치기 (Joining key columns on an index)

 

먼저 df_left_2 는 'KEY' 를 가진 DataFrame으로 만들고, df_right_2는 index를 가진 DataFrame으로 만든 후에 이 둘을 'KEY'와 index를 혼합해서 사용해서 합쳐보겠습니다.

 

 

# making DataFrame 

In [15]: df_left_2 = DataFrame({'KEY': ['K0', 'K1', 'K2', 'K3'],

    ...: 'A': ['A0', 'A1', 'A2', 'A3'],

    ...: 'B': ['B0', 'B1', 'B2', 'B3']})

    ...:



In [16]: df_right_2 = DataFrame({'C': ['C2', 'C3', 'C4', 'C5'],

    ...: 'D': ['D2', 'D3', 'D4', 'D5']},

    ...: index=['K2', 'K3', 'K4', 'K5'])

    ...:



In [17]: df_left_2  # with 'KEY'

Out[17]:

    A   B KEY
0  A0  B0  K0
1  A1  B1  K1
2  A2  B2  K2
3  A3  B3  K3



In [18]: df_right_2  # with 'index'

Out[18]:

     C   D
K2  C2  D2
K3  C3  D3
K4  C4  D4
K5  C5  D5

 

 

 

 

이제 Key와 index를 혼합해서 두 DataFrame을 합쳐보겠습니다.

 

 

# joining key columns on an index

# way 1 : pd.merge()

In [19]: pd.merge(df_left_2, df_right_2,

    ...: left_on='KEY', right_index=True,

    ...: how='left')

    ...:

Out[19]:

    A   B KEY    C    D
0  A0  B0  K0  NaN  NaN
1  A1  B1  K1  NaN  NaN
2  A2  B2  K2   C2   D2
3  A3  B3  K3   C3   D3

 

 

# way 2 : join()

In [20]: df_left_2.join(df_right_2, on='KEY', how='left')

Out[20]:

    A   B KEY    C    D
0  A0  B0  K0  NaN  NaN
1  A1  B1  K1  NaN  NaN
2  A2  B2  K2   C2   D2
3  A3  B3  K3   C3   D3

 

 

 

이상으로 DataFrame을 index 기준으로 합치는 방법에 대한 소개를 마치겠습니다.



출처: https://rfriend.tistory.com/259 [R, Python 분석과 프로그래밍의 친구 (by R Friend)]

SyntaxError: invalid syntax (<ipython-input-23-12e2df6d6a89>, line 4)